In [1]:
import numpy as np
import pandas as pd
import time
import datetime
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [2]:
features = pd.read_csv('./features.csv', index_col='match_id')

X = features.loc[:,'start_time':'dire_first_ward_time'] 
y_all = features.loc[:,'duration':'barracks_status_dire']

num = X.count().to_frame()
X = StandardScaler().fit_transform(X.fillna(0)) #замена пропусков на 0 и масштабирование
y = y_all.loc[:,'radiant_win'] #целевая переменная -- 'radiant_win'

E:\Apps\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Apps\Anaconda\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


**1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?**

In [3]:
kf = KFold(n_splits=5, shuffle = True) #генератор разбиений для кросс-валидации по 5 блокам
cpow = range(-5,5)
C = []
score = []
for i in cpow:
    C.append(10 ** i)
for c in C:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(penalty='l2', solver = 'lbfgs', C = c)
    res = np.mean(cross_val_score(clf, X, y, cv = kf, scoring = 'roc_auc'))
    score.append(res)
    print('При C = ' + str(c) + ' качество = ' + str(res) + ', затраченное время: ' + str(datetime.datetime.now() - start_time))
maxscore = max(score)
maxscorei = score.index(maxscore)
print('Лучший результат: ' + str(maxscore) + ' при С = ' + str(C[maxscorei]))

При C = 1e-05 качество = 0.6951179312487861, затраченное время: 0:00:01.772970
При C = 0.0001 качество = 0.7111806156556447, затраченное время: 0:00:01.921383
При C = 0.001 качество = 0.7161807344475418, затраченное время: 0:00:02.890072
При C = 0.01 качество = 0.7162375194444629, затраченное время: 0:00:03.878756
При C = 0.1 качество = 0.7163777898535342, затраченное время: 0:00:03.992718
При C = 1 качество = 0.7163069190006354, затраченное время: 0:00:04.092686
При C = 10 качество = 0.7162376718379526, затраченное время: 0:00:04.153667
При C = 100 качество = 0.7162880809351589, затраченное время: 0:00:04.001716
При C = 1000 качество = 0.7163918109388765, затраченное время: 0:00:03.872757
При C = 10000 качество = 0.7163161802400861, затраченное время: 0:00:04.075693
Лучший результат: 0.7163918109388765 при С = 1000


Наибольшее значение метрики AUC-ROC(0.71607) получилось при C = 0.01. Этот показатель выше чем при использовании градиентного бустинга(AUC-ROC = 0.6896).
Логистическая регрессия работает в ~10 раз быстрее градиентного бустинга.

**2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?**

In [7]:
X = features.loc[:,'start_time':'dire_first_ward_time'] 
columns = ['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero']
X.drop(columns, inplace=True, axis=1)
X = StandardScaler().fit_transform(X.fillna(0)) #замена пропусков на 0 и масштабирование
X_heroes = features.loc[:,columns] 
X_heroes.drop('lobby_type', inplace=True, axis=1) 

E:\Apps\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Apps\Anaconda\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [5]:
kf = KFold(n_splits=5, shuffle = True) #генератор разбиений для кросс-валидации по 5 блокам
cpow = range(-5,5)
C = []
score = []
for i in cpow:
    C.append(10 ** i)
for c in C:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(penalty='l2', solver = 'lbfgs', C = c)
    res = np.mean(cross_val_score(clf, X, y, cv = kf, scoring = 'roc_auc'))
    score.append(res)
    print('При C = ' + str(c) + ' качество = ' + str(res) + ', затраченное время: ' + str(datetime.datetime.now() - start_time))
maxscore = max(score)
maxscorei = score.index(maxscore)
print('Лучший результат: ' + str(maxscore) + ' при С = ' + str(C[maxscorei]))

При C = 1e-05 качество = 0.6950639641619949, затраченное время: 0:00:01.770431
При C = 0.0001 качество = 0.7113744525046666, затраченное время: 0:00:02.077334
При C = 0.001 качество = 0.7160444589202042, затраченное время: 0:00:03.179996
При C = 0.01 качество = 0.7163189711449724, затраченное время: 0:00:04.640495
При C = 0.1 качество = 0.7166367864368306, затраченное время: 0:00:06.049565
При C = 1 качество = 0.7164097550101586, затраченное время: 0:00:04.742900
При C = 10 качество = 0.7164453487813383, затраченное время: 0:00:04.480561
При C = 100 качество = 0.7164078546893528, затраченное время: 0:00:03.841768
При C = 1000 качество = 0.7164350762379791, затраченное время: 0:00:04.571533
При C = 10000 качество = 0.7161762469961173, затраченное время: 0:00:05.623195
Лучший результат: 0.7166367864368306 при С = 0.1


После удаления категориальных признаков качество работы алгоритма незначительно увеличилось(на ~0.00006). Это связано с тем, что алгоритм считает категориальные признаки как числовые.

**3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).**

In [8]:
heroes = pd.read_csv('./dictionaries/heroes.csv')
heroes_N = len(heroes)
print('В игре ' + str(heroes_num) + ' героев')

В игре 112 героев


**4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.**

In [17]:
X_pick = np.zeros((features.shape[0], heroes_N))

for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
X_pick.shape


In [22]:
X_pick = pd.DataFrame(X_pick)
X = pd.DataFrame(X)
X = pd.concat([X, X_pick], axis=1)


**5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?**

In [34]:
kf = KFold(n_splits=5, shuffle = True) #генератор разбиений для кросс-валидации по 5 блокам
cpow = range(-5,5)
C = []
score = []
for i in cpow:
    C.append(10 ** i)
for c in C:
    start_time = datetime.datetime.now()
    clf = LogisticRegression(penalty='l2', solver = 'sag', C = c)
    res = np.mean(cross_val_score(clf, X, y, cv = kf, scoring = 'roc_auc'))
    score.append(res)
    print('При C = ' + str(c) + ' качество = ' + str(res) + ', затраченное время: ' + str(datetime.datetime.now() - start_time))
maxscore = max(score)
maxscorei = score.index(maxscore)
print('Лучший результат: ' + str(maxscore) + ' при С = ' + str(C[maxscorei]))
minscore = min(score)
minscorei = score.index(minscore)
print('Худший результат: ' + str(minscore) + ' при С = ' + str(C[minscorei]))

При C = 1e-05 качество = 0.7025531550961376, затраченное время: 0:00:18.903920
При C = 0.0001 качество = 0.7308172789713854, затраченное время: 0:00:22.027931
При C = 0.001 качество = 0.7489648358182667, затраченное время: 0:00:25.456830
При C = 0.01 качество = 0.751877924107207, затраченное время: 0:00:31.611855
При C = 0.1 качество = 0.7519566422623637, затраченное время: 0:01:09.888514
При C = 1 качество = 0.7518495422486142, затраченное время: 0:01:10.129992
При C = 10 качество = 0.751755218373682, затраченное время: 0:01:25.153004
При C = 100 качество = 0.7517784252435108, затраченное время: 0:01:25.236894
При C = 1000 качество = 0.7519466124448673, затраченное время: 0:01:20.615888
При C = 10000 качество = 0.7517941462416203, затраченное время: 0:01:14.237111
Лучший результат: 0.7519566422623637 при С = 0.1
Худший результат: 0.7025531550961376 при С = 1e-05


Добавление "мешка слов" улучшило качество алгоритма(0.7520 при С = 0.1), так как теперь информация о выбранных героях используется корректно. 

**6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).**

In [28]:
test = pd.read_csv('./features_test.csv', index_col='match_id')
X_test = test.loc[:,'start_time':'dire_first_ward_time'] 
columns = ['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero']
X_test.drop(columns, inplace=True, axis=1)
X_test = StandardScaler().fit_transform(X_test.fillna(0)) #замена пропусков на 0 и масштабирование
X_test_heroes = test.loc[:,columns] 
X_test_heroes.drop('lobby_type', inplace=True, axis=1) 
X_test_pick = np.zeros((test.shape[0], heroes_N))

for i, match_id in enumerate(test.index):
    for p in range(5):
        X_test_pick[i, test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_test_pick[i, test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1
X_test_pick.shape
X_test_pick = pd.DataFrame(X_test_pick)
X_test = pd.DataFrame(X_test)
X_test = pd.concat([X_test, X_test_pick], axis=1)

E:\Apps\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
E:\Apps\Anaconda\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [33]:
c = 0.1
clf = LogisticRegression(penalty='l2', solver = 'sag', C = c)
clf.fit(X, y)
y_test = clf.predict_proba(X_test)[:, 1]
y_test

array([0.82516202, 0.75787021, 0.18742683, ..., 0.23447281, 0.62684192,
       0.42638481])

# Итог

**1. Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?**

Наибольшее значение метрики AUC-ROC(0.71607) получилось при C = 0.01. Этот показатель выше чем при использовании градиентного бустинга(AUC-ROC = 0.6896). Логистическая регрессия работает в ~10 раз быстрее градиентного бустинга.

**2. Как влияет на качество логистической регрессии удаление категориальных признаков (укажите новое значение метрики качества)? Чем вы можете объяснить это изменение?**

После удаления категориальных признаков качество работы алгоритма незначительно увеличилось(на ~0.00006). Это связано с тем, что алгоритм считает категориальные признаки как числовые.

**3. Сколько различных идентификаторов героев существует в данной игре?** В игре 112 героев.

**4. Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?**

Добавление "мешка слов" улучшило качество алгоритма(0.7520 при С = 0.1), так как теперь информация о выбранных героях используется корректно. 

**5. Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?**

Лучший результат: 0.7520 при С = 0.1
Худший результат: 0.7026 при С = 1e-05